In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/파일위치/pan_qna_kkm_w2v.csv')
sample = pd.read_csv('/content/drive/MyDrive/파일위치/pan_qna_trans_sample.csv')
df

In [ ]:
#샘플 뽑기
trans_question =[df['trans_question'][i] for i in sample['index']]
answer = [df['answer'][i] for i in sample['index']]
emd = [df['trans_q_embedding'][i] for i in sample['index']]

In [ ]:
# 역번역 질문을 공백 기준으로 나눠 리스트로 변환
##그 과정에서 오류가 나기도 해서 예외처리 해줌
t_question = []
a_pass = []
e_list = []
for i in range(len(trans_question)):
  try:
    trans_question[i].split(' ')
    t_question.append(trans_question[i])
    a_pass.append(answer[i])
    e_list.append(emd[i])
  except:
    pass
print(len(t_question))

10000


In [ ]:
question = [q.split(' ') for q in df['question']]
print(question[:3])

[['개인', '금융', '기관', '자금', '대출', '걸', '나', '대출금', '상환', '일련', '사법', '상', '계약', '아야', ''], ['종전', '규정', '허가', '자', '신', '규정', '적용', '매립', '목적', '변경', '대상', '알', '니', '라는', '이유', '행하', '부당', ''], ['군', '복무', '중', '질병', '발병', '악화', '공상', '판정', '전역', '이에', '위원', '회의', '심의', '도', '군복무', '중', '걸', '', '', '질병', '들', '', '', '발병', '치료', '사실', '인정', '다', '면', '행정청', '의무', '기록', '검토', '행하', '위법', '']]


In [ ]:
for l in t_question:
    while True:
        try:
            l.remove('')
        except:
            break
print(t_question[:3])

[['구', '지방', '세법', '규정', '주주', '때', '니까'], ['형사', '소송법', '형사', '절차', '규칙', '정의', '방법', '절차', '위반', '피고', '진술', '설명', '문서', '실제', '진정', '증명', '습니까'], ['합법적', '인', '통지', '후', '도', '의장', '이사회', '참석', '니까']]


In [ ]:
for l in question:
    while True:
        try:
            l.remove('')
        except:
            break
print(question[:3])

[['개인', '금융', '기관', '자금', '대출', '걸', '나', '대출금', '상환', '일련', '사법', '상', '계약', '아야'], ['종전', '규정', '허가', '자', '신', '규정', '적용', '매립', '목적', '변경', '대상', '알', '니', '라는', '이유', '행하', '부당'], ['군', '복무', '중', '질병', '발병', '악화', '공상', '판정', '전역', '이에', '위원', '회의', '심의', '도', '군복무', '중', '걸', '질병', '들', '발병', '치료', '사실', '인정', '다', '면', '행정청', '의무', '기록', '검토', '행하', '위법']]


In [ ]:
def str_to_vector(emd):
    res = emd.split(' ')
    #print(emd)
    #print(len(r))
    r = []
    for a in res:
        if len(a) >2:
            r.append(a)
    if len(r) != 100:
        print(len(r))
    for i in range(len(r)):
        r[i] = r[i].replace('[', '')
        r[i] = r[i].replace(']', '')
        r[i] = r[i].replace('\n', '')
        r[i] = float(r[i])
    return np.array(r)

In [ ]:
import numpy as np
emd = [str_to_vector(e) for e in e_list] #역번역 질문 만 개에 대한 임베딩 리스트
emd2 = [str_to_vector(e) for e in df['embedding']] #전체 질문에 대한 임베딩 리스트
#print(emd[0])

In [ ]:
from numpy import dot
from numpy.linalg import norm
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
from scipy.stats import pearsonr

def pearson_similarity(a, b):
    return np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

In [ ]:
import numpy as np

def dist(x,y):
    return np.sqrt(np.sum((x-y)**2))

In [ ]:
from math import *

def jaccard_similarity(x, y):
    intersection_cardinality = set(x).intersection(set(y))
    union_cardinality = set(x).union(set(y))

    return len(intersection_cardinality) / len(union_cardinality)

In [ ]:
#코사인 유사도와 피어슨 유사도, 유클리드(dist) 이용
##유클리드 이용 시 reverse=False로
def return_answer(i):
    embedding = emd[i]
    #print(embedding[0])

    #유사도 계산 및 정렬
    sim_scores = [[k, pearson_similarity(emd2[k], embedding)] for k in range(len(source))]
    sim_scores.sort(key=lambda x: x[1], reverse=True) #sim_scores의 각 리스트 중 두번째 요소를 정렬 기준으로
    #print(sim_scores[:10])

    #상위 다섯 개 저장(중복 제거)
    result = []
    for sim in sim_scores:
        a = df['answer'][sim[0]]
        if a not in result:
            result.append(a)
        else:
            #print(a)
            pass
        if len(result) == 5:
            break
    return result

In [ ]:
#자카드 유사도 이용
def return_answer(q):

    #유사도 계산 및 정렬
    sim_scores = [[k, jaccard_similarity(question[k], q)] for k in range(len(source))]
    sim_scores.sort(key=lambda x: x[1], reverse=True) #sim_scores의 각 리스트 중 두번째 요소를 정렬 기준으로
    #print(sim_scores[:10])

    #상위 다섯 개 저장(중복 제거)
    result = []
    for sim in sim_scores:
        a = answer[sim[0]]
        if a not in result:
            result.append(a)
            print(sim[1])
        else:
            #print(a)
            pass
        if len(result) == 5:
            break
    return result

In [ ]:
from tqdm import tqdm
n = len(question)
check = 0
result1 = [0]*n
result2 = [0]*n
result3 = [0]*n
result4 = [0]*n
result5 = [0]*n
for i in tqdm(range(10000)):
    if i % 500==0:
        print(i)
        print(check)
        df = {'answer(실제 답)':answer, 'result1':result1, 'result2':result2, 'result3':result3, 'result4':result4, 'result5':result5}
        df = pd.DataFrame(df)
        df.head()
        df.to_csv('/content/drive/MyDrive/파일위치/pan_kkm_jaccard_no_stopwords_sample_result.csv', index=False)
    r = return_answer(i)
    if answer[i] in r:
        check += 1
    result1[i] = r[0]
    result2[i] = r[1]
    result3[i] = r[2]
    result4[i] = r[3]
    result5[i] = r[4]

print('\n\n\n\n\n최종 결과\n', n, '\n', check)
print(check/n)
df = {'index':'answer(실제 답)':answer, 'result1':result1, 'result2':result2, 'result3':result3, 'result4':result4, 'result5':result5}
df = pd.DataFrame(df)
df.to_csv('/content/drive/MyDrive/파일위치/pan_kkm_jaccard_no_stopwords_sample_result.csv', index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/파일위치/pan_sbert_jac_n_cos_sample_result.csv')
answer = df['answer(실제 답)']
#df['result'][0]
result = []
for i in range(len(df)):
  result.append([df['result1'][i], df['result2'][i], df['result3'][i], df['result4'][i], df['result5'][i]])

"['증여세의 부과요건인 재산 증여는 과세 관청이 입증해야 하며, 재산 취득 당시 직업과 재력이 있는 경우 그 출처를 명시하지 못해도 특별한 사정이 없는 한 증여받은 것이라고 할 수 없으나, 직업이나 재력이 없는 사람이 재산의 취득자금 입증을 하지 못하고 직계 존속이 재력이 있는 경우 증여로 추정되며, 이를 번복하기 위해서는 별도의 재산취득자금 출처를 입증해야 한다. 소급과세는 그 효력발생 전에 종결한 과세요건사실에 변경된 법령 등을 적용할 수 없는 것으로, 법규 등의 변경으로 과거에는 과세대상이 되지 않던 것에 대하여 부과처분 한 것에 대해 소급과세나 재산권의 부당침해라고 할 수 없다. 또한 상속세법 시행령 제38조는 수증자의 증여세 납부의무에 대하여 증여자의 증여세 납부책임이 인정되는 경우를 구체적으로 열거하여 둔 것으로서 무효가 아니다.', '지방세법 제105조 제6항의 입법취지와 같은 법 제111조 제4항 등을 종합하여 고려하면, 이는 과점주주의 주식취득에 대한 납세의무의 폭을 보다 강화시킨 것으로서 주식취득으로 비로소 과점주주가 된 경우뿐만 아니라 과점주주가 된 후 추가로 주식을 취득한 경우도 모두 포함하여 과세요건이 되는 것임을 규정한 취지라고 해석함이 상당하며, 따라서 법인의 설립, 증자 등으로 인하여 최초로 과점주주가 된 자의 주식 추가취득에 대하여 그 취득일에 과점주주가 소유하고 있는 전주식을 취득한 것으로 보아 취득세를 부과하는 같은법시행령 제78조 제1항은 이와 같은 모법의 규정을 분명히 밝힌 데 불과하여 그 위임범위 내에 있다.', '법인의 운영을 실질적으로 지배할 수 있는 지위라 함은 실제 법인의 경영지배를 통하여 법인의 부동산 등의 재산을 사용·수익하거나 처분하는 등의 권한을 행사하였을 것을 요구하는 것은 아니고, 소유하고 있는 주식에 관하여 의결권행사 등을 통하여 주주권을 실질적으로 행사할 수 있는 지위에 있으면 족하다.', '구 지방세법(2006. 12. 30. 법률 제8147호로 개정되기 전의 것, 이하 같다) 제22조 제2호, 

In [ ]:
#첫번째와 일치하는 개수
print(len(result))
print(result[0])
c =0
for i in range(len(result)):
  if result[i][0] == summ_pass[i]:
    c+=1
print('맞힌 개수: ', c)

9826
['증여세의 부과요건인 재산 증여는 과세 관청이 입증해야 하며, 재산 취득 당시 직업과 재력이 있는 경우 그 출처를 명시하지 못해도 특별한 사정이 없는 한 증여받은 것이라고 할 수 없으나, 직업이나 재력이 없는 사람이 재산의 취득자금 입증을 하지 못하고 직계 존속이 재력이 있는 경우 증여로 추정되며, 이를 번복하기 위해서는 별도의 재산취득자금 출처를 입증해야 한다. 소급과세는 그 효력발생 전에 종결한 과세요건사실에 변경된 법령 등을 적용할 수 없는 것으로, 법규 등의 변경으로 과거에는 과세대상이 되지 않던 것에 대하여 부과처분 한 것에 대해 소급과세나 재산권의 부당침해라고 할 수 없다. 또한 상속세법 시행령 제38조는 수증자의 증여세 납부의무에 대하여 증여자의 증여세 납부책임이 인정되는 경우를 구체적으로 열거하여 둔 것으로서 무효가 아니다.', '수입물품인 프로젝터가 관세율표의 제8528.61호로 분류되기 위해서는 수입신고 당시의 구조나 기능, 특성 등에 비추어 자동자료처리시스템에 전용 또는 주로 사용되는 것이어야 하고, 그렇지 않은 경우에는 관세율표의 제8528.69호로 분류되어야 한다.', '지방세법 제105조 제6항의 입법취지와 같은 법 제111조 제4항 등을 종합하여 고려하면, 이는 과점주주의 주식취득에 대한 납세의무의 폭을 보다 강화시킨 것으로서 주식취득으로 비로소 과점주주가 된 경우뿐만 아니라 과점주주가 된 후 추가로 주식을 취득한 경우도 모두 포함하여 과세요건이 되는 것임을 규정한 취지라고 해석함이 상당하며, 따라서 법인의 설립, 증자 등으로 인하여 최초로 과점주주가 된 자의 주식 추가취득에 대하여 그 취득일에 과점주주가 소유하고 있는 전주식을 취득한 것으로 보아 취득세를 부과하는 같은법시행령 제78조 제1항은 이와 같은 모법의 규정을 분명히 밝힌 데 불과하여 그 위임범위 내에 있다.', '구 소득세법시행규칙 제67조 제1, 2호는 각 (가)항에 규정된 "매매차익"의 의미와 각 (나)항에 규정된 "매매차익"의 의미가 동일하지 않으나, 그 

In [ ]:
#다섯 개 안에 존재하는 개수
print(len(result))
c =0
for i in range(len(result)):
  if summ_pass[i] in result[i]:
    c+=1
print('맞힌 개수: ', c)

9826
맞힌 개수:  8294
